# Installations

In [48]:
!pip install matplotlib

# Imports and Sparksession


In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import os
from pyspark.sql.functions import when, col
from pyspark.ml.feature import StandardScaler
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
import matplotlib.pyplot as plt
from pyspark.ml.feature import StringIndexer, OneHotEncoder
import numpy as np
import pandas as pd
from pyspark.ml.stat import Correlation
import seaborn as sns



In [2]:

# spark = SparkSession.builder.appName("accidents").getOrCreate()

# spark = SparkSession.builder \
#     .appName("accidents") \
#     .config("spark.driver.memory", "8g") \
#     .config("spark.executor.memory", "8g") \
#     .config("spark.driver.maxResultSize", "4g") \
#     .getOrCreate()
   
spark = (
    SparkSession.builder
    .appName("accidents")
    .master("spark://spark-master:7077")    # usa davvero il cluster
    .config("spark.driver.memory", "6g")     # Jupyter driver
    .config("spark.driver.maxResultSize", "4g")  
    .config("spark.executor.memory", "6g")   # ogni executor usa 6GB
    .config("spark.executor.cores", "2")     # corrisponde ai tuoi worker
    .config("spark.executor.instances", "2") # 1 executor per worker
    .config("spark.sql.shuffle.partitions", "200") 
    .config("spark.default.parallelism", "200")
    .getOrCreate()
)
 

In [6]:
accidents = spark.read.csv("datasets/US_Accidents_March23.csv", header=True, inferSchema=True)

Py4JJavaError: An error occurred while calling o39.read.
: java.lang.IllegalStateException: LiveListenerBus is stopped.
	at org.apache.spark.scheduler.LiveListenerBus.addToQueue(LiveListenerBus.scala:92)
	at org.apache.spark.scheduler.LiveListenerBus.addToStatusQueue(LiveListenerBus.scala:75)
	at org.apache.spark.sql.internal.SharedState.<init>(SharedState.scala:115)
	at org.apache.spark.sql.SparkSession.$anonfun$sharedState$1(SparkSession.scala:143)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sharedState$lzycompute(SparkSession.scala:143)
	at org.apache.spark.sql.SparkSession.sharedState(SparkSession.scala:142)
	at org.apache.spark.sql.SparkSession.$anonfun$sessionState$2(SparkSession.scala:162)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sessionState$lzycompute(SparkSession.scala:160)
	at org.apache.spark.sql.SparkSession.sessionState(SparkSession.scala:157)
	at org.apache.spark.sql.DataFrameReader.<init>(D

# EDA

## show columns and initial ds

In [52]:
#accidents.show(5)


In [53]:
#print(accidents.columns)

## plot labels (severity distribution)

In [54]:

'''
# 1 Raggruppa severity (Spark → veloce su 7M righe)
severity_counts = (
    accidents.groupBy("severity")
             .count()
             .orderBy("severity")
)

# 2️ Converte in Pandas (poche righe → ok)
pdf = severity_counts.toPandas()

# 3️Calcolo percentuali
pdf["percent"] = pdf["count"] / pdf["count"].sum() * 100

'''


'\n# 1 Raggruppa severity (Spark → veloce su 7M righe)\nseverity_counts = (\n    accidents.groupBy("severity")\n             .count()\n             .orderBy("severity")\n)\n\n# 2️ Converte in Pandas (poche righe → ok)\npdf = severity_counts.toPandas()\n\n# 3️Calcolo percentuali\npdf["percent"] = pdf["count"] / pdf["count"].sum() * 100\n\n'

In [55]:
'''print(pdf["percent"])
# 4️⃣ Grafico a torta
plt.figure(figsize=(10, 8))
plt.pie(
    pdf["count"],
    labels=pdf["severity"],
    autopct=lambda p: f"{p:.1f}%",    # percentuali con 1 decimale
    startangle=90,
    pctdistance=0.8
)
plt.title("Distribuzione percentuale di 'severity'")
plt.axis("equal")  # mantiene il cerchio perfettamente rotondo
plt.show()'''

'print(pdf["percent"])\n# 4️⃣ Grafico a torta\nplt.figure(figsize=(10, 8))\nplt.pie(\n    pdf["count"],\n    labels=pdf["severity"],\n    autopct=lambda p: f"{p:.1f}%",    # percentuali con 1 decimale\n    startangle=90,\n    pctdistance=0.8\n)\nplt.title("Distribuzione percentuale di \'severity\'")\nplt.axis("equal")  # mantiene il cerchio perfettamente rotondo\nplt.show()'

## Fill NaN with median

In [56]:
mean_temp = accidents.select(F.mean("Temperature(F)")).first()[0]

accidents = accidents.fillna({"Temperature(F)": mean_temp})

In [57]:
weather_numeric = ['Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)','Precipitation(in)']
    
for col in weather_numeric:
    mean = accidents.select(F.mean(col)).first()[0]
    accidents = accidents.fillna({col: mean})


## Drop unuseful columns

End_* columns are removed because aren't useful to predict the severity (can be defined only afterwards the incident)
Weather_Timestamp is a value near the initial timestamp

In [58]:
#accidents = accidents.drop('Astronomical_Twilight', 'Nautical_Twilight','Country')
accidents = accidents.drop('ID', "Source",'Description', 'Distance', 'End_Time', 'Distance(mi)', 'End_Lng', 'End_Lat', 'Duration',"Weather_Timestamp")

## define categorical features

In [59]:
cat_names = ['Street', 'City', 'County', 'State', "Zipcode",
             'Timezone', 'Airport_Code', 'Wind_Direction',
              'Weather_Condition','Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight',
            "Sunrise_Sunset","Country","Turning_Loop","Amenity", "Bump", "Crossing", "Give_Way", "Junction", "No_Exit", "Railway", 
            "Roundabout", "Station","Stop","Traffic_Calming",
             "Traffic_Signal"]

## analyze categorical features

In [60]:
'''
print("Unique count of categorical features:")
for col in cat_names:
    count_unique = accidents.select(F.countDistinct(col)).first()[0]
    print(f"{col}: {count_unique}")'''

'\nprint("Unique count of categorical features:")\nfor col in cat_names:\n    count_unique = accidents.select(F.countDistinct(col)).first()[0]\n    print(f"{col}: {count_unique}")'

## Drop categorical with 1 unique value

In [61]:
accidents = accidents.drop("Country","Turning_Loop")
cat_names.remove("Country")
cat_names.remove("Turning_Loop")


## simplify wind direction 
from 24 values to 10

In [62]:
'''print("Wind Direction: ")
accidents.select("Wind_Direction").distinct().show()'''


'print("Wind Direction: ")\naccidents.select("Wind_Direction").distinct().show()'

In [63]:
wind_dir_mapping = {
    "North": 'N',
    'NNE': 'NE',
    'NE': 'NE',
    'ENE': 'E',
    'East': 'E',
    'ESE': 'SE',
    'SE': 'SE',
    'SSE': 'S',
    'South': 'S',
    'SSW': 'SW',
    'SW': 'SW',
    'WSW': 'W',
    'West': 'W',
    'WNW': 'NW',
    'NW': 'NW',
    'NNW': 'NW',
    "Calm":"CALM",
    "Variable":"VAR"
}
accidents = accidents.replace(wind_dir_mapping, subset=['Wind_Direction'])
print("Wind Direction after simplification: ")
#accidents.select("Wind_Direction").distinct().show()

Wind Direction after simplification: 


## simplify wehater_condition

In [64]:
#accidents.select("Weather_Condition").distinct().show(150)

In [65]:
patterns = {
    "Clear": "Clear",
    "Cloud": "Cloud|Overcast",
    "Rain": "Rain|storm",
    "Heavy_Rain": "Heavy Rain|Rain Shower|Heavy T-Storm|Heavy Thunderstorms",
    "Snow": "Snow|Sleet|Ice",
    "Heavy_Snow": "Heavy Snow|Heavy Sleet|Heavy Ice Pellets|Snow Showers|Squalls",
    "Fog": "Fog"
}

# inizia con colonna vuota
expr = F.lit(None)

for category, pattern in patterns.items():
    expr = F.when(
        F.col("Weather_Condition").rlike(f"(?i){pattern}"), 
        category
    ).otherwise(expr)
    
accidents = accidents.withColumn("Weather_Condition", expr)
#accidents.select("Weather_Condition").distinct().show(150)

## Conversione timestamps

In [66]:
accidents = accidents.withColumn(
    "Start_Time_parsed_str",
    F.substring(F.col("Start_Time"), 1, 19)
)

# Converte in timestamp usando il formato specifico
accidents = accidents.withColumn(
    "Start_Time_parsed",
    F.to_timestamp("Start_Time_parsed_str", "yyyy-MM-dd HH:mm:ss")
)

accidents.select("Start_Time_parsed").show(10, truncate=False)

+-------------------+
|Start_Time_parsed  |
+-------------------+
|2016-02-08 05:46:00|
|2016-02-08 06:07:59|
|2016-02-08 06:49:27|
|2016-02-08 07:23:34|
|2016-02-08 07:39:07|
|2016-02-08 07:44:26|
|2016-02-08 07:59:35|
|2016-02-08 07:59:58|
|2016-02-08 08:00:40|
|2016-02-08 08:10:04|
+-------------------+
only showing top 10 rows



### Gestione date con nuovi dati estrapolati

In [67]:
# -----------------------------
# 1) Date features
# -----------------------------
accidents = accidents.withColumn("Hour",  F.hour("Start_Time_parsed") + 1)
accidents = accidents.withColumn("Day",   F.dayofweek("Start_Time_parsed"))  # Spark: 1 = Sunday
accidents = accidents.withColumn("Month", F.month("Start_Time_parsed"))
accidents = accidents.withColumn("Year",  F.year("Start_Time_parsed"))

# -----------------------------
# 4) Part_of_Day (cut)
# -----------------------------
accidents = accidents.withColumn(
    "Part_of_Day",
    F.when(F.col("Hour").between(1, 6),  "Night")
     .when(F.col("Hour").between(7, 12), "Morning")
     .when(F.col("Hour").between(13, 18),"Afternoon")
     .otherwise("Evening")
)

# -----------------------------
# 5) Sunrise_Sunset (Day/Night)
# -----------------------------
accidents = accidents.withColumn(
    "Sunrise_Sunset",
    F.when(F.col("Hour").between(1, 16), "Day")
     .otherwise("Night")
)

# -----------------------------
# 6) Show results
# -----------------------------
accidents.select("Hour", "Day", "Month", "Year", "Part_of_Day", "Sunrise_Sunset").show(10)


+----+---+-----+----+-----------+--------------+
|Hour|Day|Month|Year|Part_of_Day|Sunrise_Sunset|
+----+---+-----+----+-----------+--------------+
|   6|  2|    2|2016|      Night|           Day|
|   7|  2|    2|2016|    Morning|           Day|
|   7|  2|    2|2016|    Morning|           Day|
|   8|  2|    2|2016|    Morning|           Day|
|   8|  2|    2|2016|    Morning|           Day|
|   8|  2|    2|2016|    Morning|           Day|
|   8|  2|    2|2016|    Morning|           Day|
|   8|  2|    2|2016|    Morning|           Day|
|   9|  2|    2|2016|    Morning|           Day|
|   9|  2|    2|2016|    Morning|           Day|
+----+---+-----+----+-----------+--------------+
only showing top 10 rows



In [68]:
accidents = accidents.drop("Start_Time_parsed","Start_Time", "Start_Time_parsed_str")
accidents=accidents.drop("Weather_Timestamp")

## Analisys of NaN

In [69]:
'''
# Numero totale di righe
total_rows = accidents.count()

# Calcolo missing per ogni colonna
missing_df = accidents.select([
    (F.count(F.when(F.col(c).isNull(), 1)) / total_rows * 100)
    .alias(c)
    for c in accidents.columns
])

# Converti in pandas per lo stesso stile tabellare
missing_pd = missing_df.toPandas().T.reset_index()

# Rename colonne per essere identici al tuo output Pandas
missing_pd.columns = ["Feature", "Missing_Percent(%)"]

# Filtra solo le colonne con missing > 0%
missing_pd = missing_pd[missing_pd["Missing_Percent(%)"] > 0]

missing_pd'''

'\n# Numero totale di righe\ntotal_rows = accidents.count()\n\n# Calcolo missing per ogni colonna\nmissing_df = accidents.select([\n    (F.count(F.when(F.col(c).isNull(), 1)) / total_rows * 100)\n    .alias(c)\n    for c in accidents.columns\n])\n\n# Converti in pandas per lo stesso stile tabellare\nmissing_pd = missing_df.toPandas().T.reset_index()\n\n# Rename colonne per essere identici al tuo output Pandas\nmissing_pd.columns = ["Feature", "Missing_Percent(%)"]\n\n# Filtra solo le colonne con missing > 0%\nmissing_pd = missing_pd[missing_pd["Missing_Percent(%)"] > 0]\n\nmissing_pd'

## Substitute NaN with "Unknown"

In [70]:
accidents = accidents.withColumn(
    "Street",
    F.coalesce(F.col("Street"), F.lit("Unknown"))
)

accidents = accidents.withColumn(
    "Zipcode",
    F.coalesce(F.col("Zipcode"), F.lit("Unknown"))
)

accidents = accidents.withColumn(
    "Weather_Condition",
    F.coalesce(F.col("Weather_Condition"), F.lit("Unknown"))
)

accidents = accidents.withColumn(
    "Wind_Direction",
    F.coalesce(F.col("Wind_Direction"), F.lit("Unknown"))
)
accidents = accidents.withColumn(
    "City",
    F.coalesce(F.col("City"), F.lit("Unknown"))
)

In [71]:
#accidents.show(5)

# Encoding delle categorical features

check cat unique values to understand type of encodings 

Unique count of categorical features:
**Frequency encoding**
- Street: 336306
- City: 13678
- County: 1871
- State: 49
- Zipcode: 825094
- Airport_Code: 2045

**One hot**
- Weather_Condition: 8

- Timezone: 4

**Numerical Encoding**
- Part_of_Day 4
- Wind_Direction: 11 (there is a sequentiality of data)

**Binary encoding**
- Sunrise_Sunset: 2
- Civil_Twilight: 2
- Nautical_Twilight: 2
- Astronomical_Twilight: 2
- Sunrise_Sunset: 2
- Amenity: 2
- Bump: 2
- Crossing: 2
- Give_Way: 2
- Junction: 2
- No_Exit: 2
- Railway: 2
- Roundabout: 2
- Station: 2
- Stop: 2
- Traffic_Calming: 2
- Traffic_Signal: 2

##  Numerical encoding

In [79]:
# 1) Crea l’indexer
weather_indexer = StringIndexer(
    inputCol="Part_of_Day",
    outputCol="Part_of_Day_idx",
    handleInvalid="keep"
)

# 2) Applica la trasformazione e aggiorna il DF
accidents = weather_indexer.fit(accidents).transform(accidents)


# 4) Mostra i valori distinti dell’indice
accidents.select("Part_of_Day_idx").distinct().show(150)

accidents=accidents.drop("Part_of_Day")


+---------------+
|Part_of_Day_idx|
+---------------+
|            0.0|
|            1.0|
|            3.0|
|            2.0|
+---------------+



In [80]:
wind_indexer = StringIndexer(
    inputCol="Wind_Direction",
    outputCol="Wind_Direction_idx",
    handleInvalid="keep"
)
accidents =wind_indexer.fit(accidents).transform(accidents)
accidents.select("Wind_Direction_idx").distinct().show(150)

accidents=accidents.drop("Wind_Direction")

+------------------+
|Wind_Direction_idx|
+------------------+
|               8.0|
|               0.0|
|               7.0|
|               1.0|
|               4.0|
|               3.0|
|               2.0|
|              10.0|
|               6.0|
|               5.0|
|               9.0|
+------------------+



## Frequency encoding 
Used for high cardinality features

In [81]:
high_card_cols = ["Street", "City", "Zipcode", "County", "Airport_Code","State"]


def freq_encode(df, col):
    freq = df.groupBy(col).count().withColumnRenamed("count", f"{col}_freq")
    df = df.join(freq, on=col, how="left")
    return df.drop(col)

for c in high_card_cols:
    accidents = freq_encode(accidents, c)


## One hot encoding 
for low cardinality features

In [82]:


small_cat = ["Weather_Condition", "Timezone"]

indexers = [
    StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep")
    for c in small_cat
]

encoders = [
    OneHotEncoder(inputCol=f"{c}_idx", outputCol=f"{c}")
    for c in small_cat
]

accidents=accidents.drop("Weather_Condition","Timezone")

## binary encoding

In [83]:

binary_cols = [
    "Sunrise_Sunset",
    "Civil_Twilight",
    "Nautical_Twilight",
    "Astronomical_Twilight",
    "Amenity",
    "Bump",
    "Crossing",
    "Give_Way",
    "Junction",
    "No_Exit",
    "Railway",
    "Roundabout",
    "Station",
    "Stop",
    "Traffic_Calming",
    "Traffic_Signal"
]

for c in binary_cols:
    accidents = accidents.withColumn(
        f"{c}_bin",
        F.when(
            F.lower(F.col(c)).isin("true", "yes", "1", "day"), 
            1
        ).otherwise(0)
    )

# Drop colonne originali
accidents = accidents.drop(*binary_cols)


In [84]:
#accidents.show(5)

## Corr matrix

|correlazione| < 0.05 → quasi zero → irrilevanti

|correlazione| < 0.01 → rumore puro → da rimuove

In [85]:
target = "severity"

correlations = []
for c in accidents.columns:
    if c != target:
        corr = accidents.stat.corr(c, target)
        correlations.append((c, corr))

corr_df = pd.DataFrame(correlations, columns=["feature", "correlation"]).dropna()
corr_df = corr_df.sort_values("correlation", ascending=False)
corr_df.head()


,feature,correlation
0,Severity,1.000000
16,Street_freq,0.129758
18,Zipcode_freq,0.086977
1,Start_Lat,0.069060
2,Start_Lng,0.052862


In [86]:
'''corr_df["abs_correlation"] = corr_df["correlation"].abs()
corr_df.sort_values("abs_correlation", ascending=False)'''

'corr_df["abs_correlation"] = corr_df["correlation"].abs()\ncorr_df.sort_values("abs_correlation", ascending=False)'

In [87]:
'''plt.figure(figsize=(20, 6))
sns.barplot(
    data=corr_df,
    x="feature",
    y="correlation",
    palette="coolwarm"
)
plt.xticks(rotation=90)
plt.title("Correlation of ALL features with Severity")
plt.tight_layout()
plt.show()
'''

'plt.figure(figsize=(20, 6))\nsns.barplot(\n    data=corr_df,\n    x="feature",\n    y="correlation",\n    palette="coolwarm"\n)\nplt.xticks(rotation=90)\nplt.title("Correlation of ALL features with Severity")\nplt.tight_layout()\nplt.show()\n'

## Drop features with corr near 0 

In [88]:
accidents=accidents.drop("Civil_Twilight_bin","Nautical_Twilight_bin","Astronomical_Twilight_bin",
                         "Traffic_Calming_bin","Bump_bin","Give_Way_bin","Day","Visibility(mi)","Roundabout_bin",
                            "Wind_Direction_idx")


In [89]:
to_drop = [
    "Civil_Twilight_bin", "Nautical_Twilight_bin", "Astronomical_Twilight_bin",
    "Traffic_Calming_bin", "Bump_bin", "Give_Way_bin",
    "Day", "Visibility(mi)", "Roundabout_bin", "Wind_Direction_idx"
]

corr_df = corr_df[~corr_df["feature"].isin(to_drop)]


In [90]:

'''plt.figure(figsize=(20, 6))
sns.barplot(
    data=corr_df,
    x="feature",
    y="correlation",
    palette="coolwarm"
)
plt.xticks(rotation=90)
plt.title("Correlation of ALL features with Severity")
plt.tight_layout()
plt.show()
corr_df.sort_values("abs_correlation", ascending=False)'''

'plt.figure(figsize=(20, 6))\nsns.barplot(\n    data=corr_df,\n    x="feature",\n    y="correlation",\n    palette="coolwarm"\n)\nplt.xticks(rotation=90)\nplt.title("Correlation of ALL features with Severity")\nplt.tight_layout()\nplt.show()\ncorr_df.sort_values("abs_correlation", ascending=False)'

# Class weights

In [ ]:
'''
counts = accidents.groupBy("severity").count().toPandas()
total = counts["count"].sum()

weights = {row["severity"]: total / row["count"] for _, row in counts.iterrows()}
print(weights)

accidents = accidents.withColumn(
    "classWeightCol",
    F.when(F.col("severity") == 1, weights[1])
     .when(F.col("severity") == 2, weights[2])
     .when(F.col("severity") == 3, weights[3])
     .when(F.col("severity") == 4, weights[4])
)'''


# Stratified Train test split

In [91]:
'''Aggiungi una colonna random per il sort'''
df = accidents.withColumn("rand", F.rand(seed=42))

In [92]:
'''Ordina per severity e rand (stratified shuffle)

Questo garantisce che le classi siano mescolate ma mantenendo proporzioni identiche'''
df = df.orderBy("severity", "rand")


In [93]:
'''Split normale (è STRATIFICATO perché il dataset è ordinato!)

Con tantissimi record per classe, questo funziona perfettamente.'''
train, test = df.randomSplit([0.7, 0.3], seed=42)


In [94]:
from collections import Counter

'''train_counts = Counter(train.select("severity").rdd.map(lambda r: r[0]).takeSample(False, 500000))
test_counts  = Counter(test.select("severity").rdd.map(lambda r: r[0]).takeSample(False, 500000))

print("TRAIN (sampled):", train_counts)
print("TEST  (sampled):", test_counts)'''


'train_counts = Counter(train.select("severity").rdd.map(lambda r: r[0]).takeSample(False, 500000))\ntest_counts  = Counter(test.select("severity").rdd.map(lambda r: r[0]).takeSample(False, 500000))\n\nprint("TRAIN (sampled):", train_counts)\nprint("TEST  (sampled):", test_counts)'

In [95]:
from pyspark.ml.feature import VectorAssembler

feature_cols = [
    "Start_Lat", "Start_Lng", "Temperature(F)", "Wind_Chill(F)", "Humidity(%)",
    "Pressure(in)", "Wind_Speed(mph)", "Precipitation(in)",
    "Hour", "Month", "Year",
    "Part_of_Day_idx", "Street_freq", "City_freq", "Zipcode_freq",
    "County_freq", "Airport_Code_freq", "State_freq",
    "Sunrise_Sunset_bin", "Amenity_bin", "Crossing_bin", "Junction_bin",
    "No_Exit_bin", "Railway_bin", "Station_bin", "Stop_bin",
    "Traffic_Signal_bin"
]

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
    handleInvalid="skip"   # <-- salta le righe che contengono null
)

train_assembled = assembler.transform(train)


# Models

In [77]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import time

## RandomForest

In [96]:
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="Severity",
    #weightCol="classWeightCol",   # <-- importantissimo
    numTrees=10,
    maxDepth=12,
    maxBins=32,
    seed=42
)

pipeline = Pipeline(stages=[
    assembler,
    rf
])

start_time = time.time()
rf_model = pipeline.fit(train)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")
with open("rf_model_time.txt", "w") as f:
    f.write(f"Training time: {end_time - start_time:.2f} seconds\n")
    f.close()


Training time: 211.56 seconds


In [47]:
predictions = rf_model.transform(test)

evaluator = MulticlassClassificationEvaluator(
    labelCol="Severity",
    predictionCol="prediction",
    metricName="f1"
)

f1 = evaluator.evaluate(predictions)
print("F1 =", f1)

acc = evaluator.setMetricName("accuracy").evaluate(predictions)
print("Accuracy =", acc)

F1 = 0.8072335776242325
Accuracy = 0.834787565031444


## Logistic Regression

In [97]:
lr = LogisticRegression(
    featuresCol="features",
    labelCol="Severity",
    maxIter=50,
    regParam=0.0,
    elasticNetParam=0.0
)

from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[
    assembler,
    lr
])

start_time = time.time()
lr_model = pipeline.fit(train)
end_time = time.time()
print(f"Training time: {end_time - start_time} seconds")  
  
with open("lr_model_time.txt", "w") as f:
    f.write(f"Training time: {end_time - start_time:.2f} seconds\n")
    f.close()

Training time: 141.05464482307434 seconds


# Evaluation

In [98]:
predictions = lr_model.transform(test)

evaluator = MulticlassClassificationEvaluator(
    labelCol="Severity",
    predictionCol="prediction",
    metricName="f1"
)

f1 = evaluator.evaluate(predictions)
print("F1 =", f1)

acc = evaluator.setMetricName("accuracy").evaluate(predictions)
print("Accuracy =", acc)

F1 = 0.7520276736954681
Accuracy = 0.8014088426121663
